In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os
import numpy as np
import pandas as pd
# import regex as re
# %pip install regex as re
import requests


In [2]:
#Set Header
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
  'referer': 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA',
  'origin': 'https://www.realtor.com'
  }

In [3]:
with requests.session() as s:
    city = 'San-Francisco_CA'
    page = 1
    end_page = 10
    url = ''
    url_list = []
    
    while page <= end_page:
        url = 'https://www.realtor.com/realestateandhomes-search/' +city+ f'/pg-{page}'
        url_list.append(url)
        page += 1
    
    request = ''
    request_list = []
    
    for url in url_list:
        request = s.get(url, headers=header)
        request_list.append(request)
    
B_soup = ''
soup_list = []
print(url_list)

for request in request_list:
    B_soup = soup(request.content, 'html.parser')
    print(request.status_code)
    soup_list.append(B_soup)

['https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-1', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-2', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-3', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-4', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-5', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-6', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-7', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-8', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-9', 'https://www.realtor.com/realestateandhomes-search/San-Francisco_CA/pg-10']
200
200
200
200
200
200
200
200
200
200


In [4]:
pd.set_option('display.width', 1000)
df_list = []
for bsobj in soup_list:
    df = pd.DataFrame()

    df['property type'] = bsobj.find_all('div', {'class': 'base__StyledType-rui__sc-108xfm0-0 iSdpGk message'})
    df['prices'] = bsobj.find_all('div', {'data-testid': 'card-price'})
    # df['beds'] = bsobj.find_all('li', { 'data-testid': 'property-meta-beds'})
    df['beds_baths_size'] = bsobj.find_all('ul', { 'data-testid': 'card-meta'})
    df['address 1'] = bsobj.find_all('div', {'data-testid': 'card-address-1'})
    df['address 2'] = bsobj.find_all('div', {'data-testid': 'card-address-2'})

    df_list.append(df)


df = pd.concat(df_list).reset_index().drop(columns='index')

In [5]:
df['address 1'] = df['address 1'].astype('str')
# df
df.loc[:,'address 1'] = df.loc[:,'address 1'].replace('<div class="truncate-line" data-testid="card-address-1">', '', regex=True)
df.loc[:,'address 1'] = df.loc[:,'address 1'].replace('</div>', '', regex=True)


# Function to remove HTML tags using BeautifulSoup
def remove_html_tags(text):
    soup_s = soup(text, 'html.parser')
    return ''.join(soup_s.stripped_strings).replace('bed', 'bed ').replace('bath', 'bath ').split('sqft')[0]

df['beds_baths_size'] = df['beds_baths_size'].astype('str')
# Apply the function to 'beds_baths_size' column
df['beds_baths_size'] = df['beds_baths_size'].apply(remove_html_tags)

df

,property type,prices,beds_baths_size,address 1,address 2
0,[House for sale],"[$107,500]",3bed 1bath,169 Raymond Ave,"[San Francisco, CA 94134]"
1,[Condo for sale],"[$361,677]","2bed 2bath 1,089",74 New Montgomery St Unit 317,"[San Francisco, CA 94105]"
2,[Condo for sale],"[$309,000]",Studio1bath 480,631 Ofarrell St Apt 416,"[San Francisco, CA 94109]"
3,[House for sale],"[$7,125,000]","4bed 3.5bath 4,815",2763 Filbert St,"[San Francisco, CA 94123]"
4,[House for sale],"[$799,999]","4bed 4bath 1,468",1883 Oakdale Ave,"[San Francisco, CA 94124]"
...,...,...,...,...,...
75,[Pending],"[$955,000]","2bed 2bath 1,170",782 15th Ave,"[San Francisco, CA 94118]"
76,[Pending],"[$1,195,000]","3bed 1,352",871 DE Haro St,"[San Francisco, CA 94107]"
77,[House for sale],"[$885,000]","4bed 1bath 1,036",864 Rolph St,"[San Francisco, CA 94112]"
78,[House for sale],"[$1,985,000]","7bed 5.5bath 2,495",62 Vernon St,"[San Francisco, CA 94132]"
